In [92]:
from gensim.models import Word2Vec
from sklearn.linear_model import LinearRegression
import numpy as np

In [93]:
# preprocess tweets
import sys, csv
sys.path.append('modules/')
from wordcount import mk_words
sentences = []
labels = []
with open('tweets.csv') as fp:
    csv_r = csv.reader(fp)
    for row in csv_r:
        try:
            score = float(row[2])
            labels.append(score)
        except:
            continue
        sentence = row[1]
        words = mk_words(sentence)
        sentences.append(words)

In [94]:
# create word2vec model
model = Word2Vec(sentences, min_count=10, size=300)

In [95]:
from functools import reduce
def ave_vectors(vector_list, size):
    sum_vector = np.zeros(size)
    for vector in vector_list:
        sum_vector += vector
    if (len(vector_list) == 0):
        length = 1
    else:
        length = len(vector_list)
    return(1.0*sum_vector / length)

def sentence_vector(words):
    #average all words in sentence to one vectorf
#     words = mk_words(sentence)
    word_vectors = []
    for word in words:
        try:
            word_vectors.append(model.wv[word])
        except KeyError:
            continue
    return ave_vectors(word_vectors, 300)
    

In [96]:
# create training and testing vector list
sentences_vec = list(map(sentence_vector, sentences))
n = len(sentences)
training_sent_arr = np.array(sentences_vec[:int(0.8*n)])
testing_sent_arr = np.array(sentences_vec[int(0.8*n):])
training_label_arr = np.array(labels[:int(0.8*n)])
testing_label_arr = np.array(labels[int(0.8*n):])
testing_label_arr.shape

(858,)

In [97]:
# create linear regressor
lr = LinearRegression()
lr.fit(training_sent_arr, training_label_arr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [98]:
res = lr.predict(testing_sent_arr)
succes = 0
for i in range(0, res.size):
    diff = (res[i] - testing_label_arr[i])**2
    if diff <= 0.001:
        succes += 1
print(succes / res.size)

0.5699300699300699


In [105]:
# make labels suitable of MLP classifier
# values of boundaries has been determined by manual analysis
boundaries =[
    0.23,
    0.1958,
    0.1680,
    0.1475
]
def get_lbl(x):
    if x>boundaries[0]:
        return 4
    elif x<boundaries[3]:
        return 0
    elif x<boundaries[1]:
        return 3
    elif x<boundaries[2]:
        return 1
    else:
        return 2

def get_vec(score):
    lbl = get_lbl(score)
    vector = [0,0,0,0,0]
    vector[lbl] = 1
    return np.array(vector)

label_vectors = [get_vec(score) for score in labels]
n = len(label_vectors)
training_label_arr = np.array(label_vectors[:int(0.8*n)])
testing_label_arr = np.array(label_vectors[int(0.2*n):])

In [110]:
# now let us try multi layer neural networks
from sklearn.neural_network import MLPClassifier
mlpr = MLPClassifier()
mlpr.fit(training_sent_arr, training_label_arr)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [114]:
mlpr.predict([testing_sent_arr[20]])

array([[0, 0, 0, 0, 0]])